In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
BASE_DIR = '/content/drive/My Drive/datasets/'

In [4]:
# %%
# !pip install ipympl
%matplotlib notebook
import pandas as pd
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns
import numpy as np
import os
# %%
# %%


In [5]:

df = pd.read_csv(os.path.join(BASE_DIR, 'WearableComputing_weight_lifting_exercises_biceps_curl_variations.csv'))
df = df.replace('', np.nan, regex=True)
df = df.replace('NA', np.nan, regex=True)
class_map = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5
}
df['classe'] = df.classe.apply(class_map.get)
df.dropna(axis=1, inplace=True)
df.shape
# %%

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,13,18,21,24,67,68,69,70,71,72,85,86,88,89,93,96,99,123,124,126,127,131,134,137) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(39242, 58)

In [6]:
df = df.sample(n=20000)
train_attrs = list(df.columns.values)            
train_attrs.pop(train_attrs.index('classe'))
train_attrs.pop(train_attrs.index('user_name'))
train_attrs.pop(train_attrs.index('raw_timestamp_part_1'))
train_attrs.pop(train_attrs.index('raw_timestamp_part_2'))
train_attrs.pop(train_attrs.index('cvtd_timestamp'))
train_attrs.pop(train_attrs.index('new_window'))
# display(train_attrs)
display(df[train_attrs].dtypes, len(train_attrs))
X, Y = df[train_attrs], df['classe']
x_train, x_test, y_train, y_test = train_test_split(X,Y, train_size=0.8)
# clf = LinearSVC()
# clf = SVC(kernel='linear', probability=False, cache_size=2000)
# clf = SVC(kernel='linear', cache_size=10000, probability=False)
# clf = SVC(kernel='linear', degree=0, C=0.1) # try degree=2,4
clf = SVC(C=16, kernel='rbf', probability=False, cache_size=4000)
# clf = SVC(kernel='poly', degree=4, class_weight={1:0.3, 2:0.2, 3:0.1, 4:0.2, 5:0.2 }) # try degree=2,4
# display(x_train)
# display(y_train)
clf.fit(x_train, y_train)



num_window                int64
roll_belt               float64
pitch_belt              float64
yaw_belt                float64
total_accel_belt          int64
gyros_belt_x            float64
gyros_belt_y            float64
gyros_belt_z            float64
accel_belt_x              int64
accel_belt_y              int64
accel_belt_z              int64
magnet_belt_x             int64
magnet_belt_y             int64
magnet_belt_z             int64
roll_arm                float64
pitch_arm               float64
yaw_arm                 float64
total_accel_arm           int64
gyros_arm_x             float64
gyros_arm_y             float64
gyros_arm_z             float64
accel_arm_x               int64
accel_arm_y               int64
accel_arm_z               int64
magnet_arm_x              int64
magnet_arm_y              int64
magnet_arm_z              int64
pitch_dumbbell          float64
yaw_dumbbell            float64
total_accel_dumbbell      int64
gyros_dumbbell_x        float64
gyros_du

52

SVC(C=16, break_ties=False, cache_size=4000, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [7]:
y_pred = clf.predict(x_test)
p,r,f,s = precision_recall_fscore_support(y_test, y_pred, labels=[1,2,3,4,5])
display('f-score = {}'.format(f))
display([round(each,2) for each in f])
# display([round(each,2) for each in r])
# display([round(each,2) for each in p])


'f-score = [0.98027696 0.94175439 0.91739422 0.91460495 0.96827586]'

[0.98, 0.94, 0.92, 0.91, 0.97]

In [8]:
from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5])
plt.figure(figsize=(15,10))
ax = sns.heatmap(cf_mat, annot=True,  fmt='g', annot_kws={"size": 20}, xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
ax.tick_params(labelsize=20)

<IPython.core.display.Javascript object>

In [9]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
lm.fit(x_train, y_train)
y_pred = lm.predict(x_test)

p,r,f,s = precision_recall_fscore_support(y_test, y_pred, labels=[1,2,3,4,5])
display([round(each,2) for each in f])

from sklearn.metrics import confusion_matrix
lm_cf_mat = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5])
plt.figure()
sns.heatmap(lm_cf_mat, annot=True)
plt.show()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.81, 0.63, 0.62, 0.63, 0.6]

<IPython.core.display.Javascript object>

In [10]:

def plot_3d(x, y, z, _df):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')

    ax.set_xlabel(x)
    ax.set_ylabel(y)
    ax.set_zlabel(z)
    
#     cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())
    sc = ax.scatter(_df[x], _df[y], _df[z], c=df['classe'])
    # legend
    plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)
#     ax.legend()
    plt.show()

# %%

In [11]:
plot_3d('roll_belt', 'yaw_belt', 'pitch_belt', df)

<IPython.core.display.Javascript object>

In [12]:
plot_3d('roll_forearm', 'yaw_forearm', 'pitch_forearm', df)

<IPython.core.display.Javascript object>

In [13]:
plot_3d('roll_arm', 'yaw_arm', 'pitch_arm', df)

<IPython.core.display.Javascript object>

In [14]:
plot_3d('total_accel_belt', 'total_accel_forearm', 'total_accel_arm', df)

<IPython.core.display.Javascript object>